# Questions?
1. Why dummy variable are creted and rest of var. are droped?


In [68]:
import pandas as pd 
from pandas import Series,DataFrame

#import numpy,seaborn,matplotlib
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
import matplotlib.pyplot as plt
%pylab

#machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [69]:
# get titanic & test csv files as a DataFrame
titanic_df=pd.read_csv(r'C:\Users\PUSHPENDRA\Desktop\career\Titanic\train.csv')
test_df=pd.read_csv(r'C:\Users\PUSHPENDRA\Desktop\career\Titanic\test.csv')

#preview the data
titanic_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [70]:
titanic_df.info()
print("---------------------------")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
---------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare 

In [71]:
# drop unnecessary columns, these columns won't be useful in analysis and prediction
titanic_df = titanic_df.drop(['PassengerId','Name','Ticket'], axis=1)
test_df    = test_df.drop(['Name','Ticket'], axis=1)

->Some features are not required for model building therefore they are removed.

In [72]:
#Embarked

# only in titanic_df, fill the two missing values with the most occurred value, which is "S".
titanic_df['Embarked']=titanic_df['Embarked'].fillna("S")

#plot
sns.factorplot('Embarked','Survived',data=titanic_df,size=4,aspect=3)
fig ,(axis1,axis2,axis3)=plt.subplots(1,3,sharex=True,sharey=True)

# sns.factorplot('Embarked',data=titanic_df,kind='count',order=['S','C','Q'],ax=axis1)
# sns.factorplot('Survived',hue="Embarked",data=titanic_df,kind='count',order=[1,0],ax=axis2)

sns.countplot(x='Embarked',data=titanic_df,ax=axis1)
sns.countplot(x='Survived',hue='Embarked',data=titanic_df,order=[1,0],ax=axis2)

# group by embarked, and get the mean for survived passengers for each value in Embarked

embark_perc=titanic_df[["Embarked","Survived"]].groupby(["Embarked"],as_index=False).mean()
sns.barplot(x="Embarked",y="Survived",data=embark_perc,order=['S','C','Q'],ax=axis3)

# Either to consider Embarked column in predictions,
# and remove "S" dummy variable, 
# and leave "C" & "Q", since they seem to have a good rate for Survival.

# OR, don't create dummy variables for Embarked column, just drop it, 
# because logically, Embarked doesn't seem to be useful in prediction.

embark_dummies_titanic= pd.get_dummies(titanic_df['Embarked'])
embark_dummies_titanic.drop(['S'],axis=1,inplace=True)

embark_dummies_test  = pd.get_dummies(test_df['Embarked'])
embark_dummies_test.drop(['S'], axis=1, inplace=True)

titanic_df=titanic_df.join(embark_dummies_titanic)
test_df=test_df.join(embark_dummies_test)

titanic_df.drop(['Embarked'],axis=1,inplace=True)
test_df.drop(['Embarked'],axis=1,inplace=True)


In [73]:
#Fare

# only for test_df, since there is a missing "Fare" values

test_df["Fare"].fillna(test_df["Fare"].median(),inplace=True)

# convert from float to int
titanic_df["Fare"]=titanic_df["Fare"].astype(int)
test_df["Fare"]=test_df["Fare"].astype(int)


# get fare for survived & didn't survive passengers 
fare_not_survived=titanic_df["Fare"][titanic_df["Survived"]==0]
fare_survived=titanic_df["Fare"][titanic_df["Survived"]==1]


# get average and std for fare of survived/not survived passengers
average_fare=DataFrame([fare_not_survived.mean(),fare_survived.mean()])
std_fare=DataFrame([fare_not_survived.std(),fare_survived.std()])

# plot
titanic_df['Fare'].plot(kind='hist',figsize=(15,3),bins=100,xlim=(0,50))

average_fare.index.names=std_fare.index.names=["Survived"]
average_fare.plot(yerr=std_fare,kind="bar",legend=False)


In [74]:
#age
# get average, std, and number of NaN values in titanic_df
average_age_titanic=titanic_df["Age"].mean()
std_age_titanic=titanic_df['Age'].std()
count_nan_age_titanic=titanic_df["Age"].isnull().sum()

# get average, std, and number of NaN values in test_df
average_age_test=test_df["Age"].mean()
std_age_test=test_df["Age"].std()
count_nan_age_test=test_df["Age"].isnull().sum()

# generate random numbers between (mean - std) & (mean + std)
rand1=np.random.randint(average_age_titanic-std_age_titanic,average_age_titanic+std_age_titanic,size=count_nan_age_titanic)
rand2=np.random.randint(average_age_test-std_age_test,average_age_test+std_age_test,size=count_nan_age_test)

# plot original Age values
# NOTE: drop all null values, and convert to int
plt.title('Original Age values - Titanic')
plt1=plt.hist(titanic_df["Age"].dropna().astype(int),bins=70)
pylab.savefig('plt1.png')
# test_df['Age'].dropna().astype(int).hist(bins=70, ax=axis1)

# fill NaN values in Age column with random values generated
titanic_df["Age"][np.isnan(titanic_df["Age"])]=rand1
test_df["Age"][np.isnan(test_df["Age"])]=rand2

# convert from float to int
titanic_df["Age"]=titanic_df["Age"].astype(int)
test_df["Age"]=test_df["Age"].astype(int)

# plot new Age Values
plt.title('New Age values - Titanic')
plt2=plt.hist(titanic_df["Age"],bins=70)
pylab.savefig('plt2.png')
#test_df["Age"].hist(bins=70,ax=axis4)


C:\Users\PUSHPENDRA\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\PUSHPENDRA\Anaconda3\lib\site-packages\ipykernel\__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
 #.... continue with plot Age column
# peaks for survived/not survived passengers by their age

facet=sns.FacetGrid(titanic_df,hue="Survived",aspect=4)
facet.map(sns.kdeplot,'Age',shade=True)
facet.set(xlim(0,titanic_df['Age'].max()))
facet.add_legend()


    

C:\Users\PUSHPENDRA\Anaconda3\lib\site-packages\statsmodels\nonparametric\kdetools.py:20: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  y = X[:m/2+1] + np.r_[0,X[m/2+1:],0]*1j


TypeError: set() takes 1 positional argument but 2 were given

In [76]:
# average survived passengers by age
fig,axis1=plt.subplots(1,1,figsize=(18,4))
average_age=titanic_df[["Age","Survived"]].groupby(["Age"],as_index=False).mean()
sns.barplot(x="Age",y="Survived",data=average_age)

In [77]:
# Cabin
# It has a lot of NaN values, so it won't cause a remarkable impact on prediction
titanic_df.drop("Cabin",axis=1,inplace=True)
test_df.drop("Cabin",axis=1,inplace=True)

In [78]:
#Family
# Instead of having two columns Parch & SibSp, 
# we can have only one column represent if the passenger had any family member aboard or not,
# Meaning, if having any family member(whether parent, brother, ...etc) will increase chances of Survival or not.

titanic_df["Family"]=titanic_df["SibSp"]+titanic_df["Parch"]
titanic_df["Family"].loc[titanic_df["Family"]>0]=1
titanic_df["Family"].loc[titanic_df["Family"]==0]=0

test_df["Family"]=test_df["SibSp"]+test_df["Parch"]
test_df["Family"].loc[test_df["Family"]>0]=1
test_df["Family"].loc[test_df["Family"]==0]=0

# drop Parch & SibSp
titanic_df.drop(["Parch","SibSp"],axis=1)
test_df.drop(["Parch","SibSp"],axis=1)

#plot
fig,(axis1,axis2)=plt.subplots(1,2,sharex=True,figsize=(10,5))

# sns.factorplot('Family',data=titanic_df,kind='count',ax=axis1)
sns.countplot(x="Family",data=titanic_df,order=[1,0],ax=axis1)

# average of survived for those who had/didn't have any family member
family_persec=titanic_df[["Family","Survived"]].groupby(["Family"],as_index=False).mean()
sns.barplot(x="Family",y="Survived",data=family_persec,order=[1,0],ax=axis2)

axis1.set_xticklabels(["With Family","Alone"],rotation=0)



C:\Users\PUSHPENDRA\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [79]:
#Sex

# As we see, children(age < ~16) on aboard seem to have a high chances for Survival.
# So, we can classify passengers as males, females, and child

def get_person(passenger):
    age,sex=passenger
    return 'child' if age<16 else  sex

titanic_df['Person'] = titanic_df[['Age','Sex']].apply(get_person,axis=1)
test_df['Person']    = test_df[['Age','Sex']].apply(get_person,axis=1)

# No need to use Sex column since we created Person column
titanic_df.drop(["Sex"],axis=1,inplace=True)
test_df.drop(["Sex"],axis=1,inplace=True)

# create dummy variables for Person column, & drop Male as it has the lowest average of survived passengers

person_dummies_titanic=pd.get_dummies(titanic_df["Person"])
person_dummies_titanic.columns=["Child","Male","Female"]
person_dummies_titanic.drop(["Male"],axis=1,inplace=True)

person_dummies_test=pd.get_dummies(test_df["Person"])
person_dummies_test.columns=["Child","Male","Female"]
person_dummies_test.drop(["Male"],axis=1,inplace=True)

titanic_df=titanic_df.join(person_dummies_titanic)
test_df=test_df.join(person_dummies_test)

fig, (axis1,axis2)=plt.subplots(1,2,figsize=(10,5))

# sns.factorplot('Person',data=titanic_df,kind='count',ax=axis1)
sns.countplot(x="Person",data=titanic_df,ax=axis1)

# average of survived for each Person(male, female, or child)
person_perc = titanic_df[["Person", "Survived"]].groupby(['Person'],as_index=False).mean()
sns.barplot(x='Person', y='Survived', data=person_perc, ax=axis2, order=['male','female','child'])

titanic_df.drop(['Person'],axis=1,inplace=True)
test_df.drop(['Person'],axis=1,inplace=True)

In [80]:
# Pclass
# sns.factorplot('Pclass',data=titanic_df,kind='count',order=[1,2,3])
sns.factorplot(x="Pclass",y="Survived",data=titanic_df,order=[1,2,3],size=5)

# create dummy variables for Pclass column, & drop 3rd class as it has the lowest average of survived passengers
pclass_dummies_titanic=pd.get_dummies(titanic_df["Pclass"])
pclass_dummies_titanic.columns=["Class_1","Class_2","Class_3"]
pclass_dummies_titanic.drop(["Class_1"],axis=1,inplace=True)

pclass_dummies_test=pd.get_dummies(test_df["Pclass"])
pclass_dummies_test.columns=["Class_1","Class_2","Class_3"]
pclass_dummies_test.drop(["Class_1"],axis=1,inplace=True)

titanic_df.drop(["Pclass"],axis=1,inplace=True)
test_df.drop(["Pclass"],axis=1,inplace=True)

titanic_df=titanic_df.join(pclass_dummies_titanic)
test_df=test_df.join(pclass_dummies_test)

In [81]:
# define training and testing sets

X_train=titanic_df.drop(["Survived"],axis=1)
Y_train=titanic_df["Survived"]
X_test=test_df.drop(["PassengerId"],axis=1).copy()

In [82]:
# Logistic Regression
logreg=LogisticRegression()

logreg.fit(X_train,Y_train)

Y_pred=logreg.predict(X_test)

logreg.score(X_train,Y_train)

0.81930415263748602

In [83]:
# Support Vector Machines
svc=SVC()

svc.fit(X_train,Y_train)

Y_pred=svc.predict(X_test)

svc.score(X_train,Y_train)

0.86756453423120095

In [87]:
#Random Forest
random_forest=RandomForestClassifier(n_estimators=100)

random_forest.fit(X_train,Y_train)

Y_pred=random_forest.predict(X_test)

random_forest.score(X_train,Y_train)

0.96745230078563416

In [84]:
#Kneighborclassifier

knn=KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train,Y_train)

Y_predict=knn.predict(X_test)

knn.score(X_train,Y_train)

0.81593714927048255

In [85]:
#Gaussian Naive Bayes
gaussian=GaussianNB()

gaussian.fit(X_train,Y_train)

Y_predict=gaussian.predict(X_test)

gaussian.score(X_train,Y_train
              )

0.79797979797979801

In [86]:
# get Correlation Coefficient for each feature using Logistic Regression
coeff_df=DataFrame(titanic_df.columns.delete(0))
coeff_df.columns=['Features']
coeff_df["Coefficient Estimate"]=pd.Series(logreg.coef_[0])

#preview
coeff_df

,Features,Coefficient Estimate
0,Age,-0.008279
1,SibSp,-0.566234
2,Parch,-0.341056
3,Fare,0.004559
4,C,0.346017
5,Q,0.239028
6,Family,0.694570
7,Child,-0.171770
8,Female,-2.773364
9,Class_2,-0.532955


In [90]:
submission=pd.DataFrame({"PassengerId":test_df["PassengerId"],
                        "Survived":Y_pred})
submission.to_csv(r'C:\Users\PUSHPENDRA\Desktop\career\Titanic\titanic.csv',index=False)